In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

CHROME_BIN = "/usr/bin/chromium-browser"
CHROME_DRIVER = '/usr/lib/chromium-browser/chromedriver'
options = Options()
options.binary_location = CHROME_BIN
options.add_argument("--disable-dev-shm-usage")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument('--headless')

browser = webdriver.Chrome(CHROME_DRIVER, options=options)
browser.get('https://sandbox.kikagaku.co.jp/blog/')

In [22]:
elems = browser.find_elements_by_class_name('stretched-link')
len(elems)

4

In [23]:
for elem in elems:
    print(elem.get_attribute('href'))

https://sandbox.kikagaku.co.jp/blog/post1.html
https://sandbox.kikagaku.co.jp/blog/post2.html
https://sandbox.kikagaku.co.jp/blog/post3.html
https://sandbox.kikagaku.co.jp/blog/post4.html


In [24]:
urls = []
for elem in elems:
    url = elem.get_attribute('href')
    urls.append(url)
    
urls

['https://sandbox.kikagaku.co.jp/blog/post1.html',
 'https://sandbox.kikagaku.co.jp/blog/post2.html',
 'https://sandbox.kikagaku.co.jp/blog/post3.html',
 'https://sandbox.kikagaku.co.jp/blog/post4.html']

In [31]:
url = elems[0].get_attribute('href')
url

'https://sandbox.kikagaku.co.jp/blog/post1.html'

In [32]:
browser.get(url)

In [33]:
elem = browser.find_element_by_class_name('newsContent_inner')
elem.text

'人工知能・機械学習領域でセミナー事業を展開する株式会社キカガク(本社：東京都千代田区、代表取締役社長：吉崎 亮介)は、2019年4月に Preferred Networks 株式会社から無料で公開された Chainer チュートリアルの企画、Web サイト制作、一部のチュートリアル記事の執筆に協力いたしました。\nキカガクで主軸として行っている社会人向けのAI領域での教育事業を大学生や学びはじめの社会人の方向けに広く提供していく予定です。日本では AI 人材育成が急務となっており、今回のチュートリアルの無料公開がこの人材の育成への第一歩となることを願っています。\n  ■Chainer チュートリアル\nChainer チュートリアルは、機械学習や深層学習に興味があり、その仕組みや使い方を理解したいと考えている大学生や社会人の方に向けて書かれた深層学習の初心者向けオンライン学習資料です。大学の授業や企業研修、商用セミナーなどで広く利用されることを想定し、誰でも無料で使用することが可能です。機械学習の勉強を進めるために必要な数学の基礎から、Python というプログラミング言語を用いたコーディングの基本、機械学習・深層学習の基礎的な理論、機械学習を応用する方法に至るまで、幅広いトピックを解説しています。\n▽ Chainer チュートリアル\nhttps://tutorials.chainer.org/ja/\n■AI 人材育成へ興味がある企業・大学の方へ\nキカガクでは多くの企業への法人研修や、東京大学・東北大学をはじめとした大学・高専への講義も行ってきました。また、一般向けの短期セミナーや長期セミナーだけでなく、製造業に特化したコースなど各企業と連携した人材育成への取り組みも進めています。AI人材の育成が急務となっている企業の方、各業界固有の知見を活かした協業や研修のご相談をお待ちしております。\n▽参考：製造業特化型データサイエンス集中コース\nhttps://dsit.kikagaku.co.jp/\n  【株式会社キカガク 会社概要】\n代表者\u3000\u3000： 吉崎 亮介\n設立\u3000\u3000\u3000： 2017年1月17日\n本社所在地： 東京都千代田区内神田3丁目2-9 SPビル6階\n事業内容\u3000： 機械学習の教育およ

In [34]:
from janome.tokenizer import Tokenizer
from gensim import corpora, matutils
import joblib

tagger = Tokenizer(wakati=True)
dictionary = joblib.load('dictionary.pkl')
clf = joblib.load('document_classifier.pkl')

In [35]:
n_words = len(dictionary)
n_words

9797

In [36]:
def preprocessing(text):
    text = text.replace('\u3000', '')
    text = text.replace('\n', '')
    words = tagger.tokenize(text)
    return words

words = preprocessing(elem.text)
words[:10]

['人工', '知能', '・', '機械', '学習', '領域', 'で', 'セミナー', '事業', 'を']

In [37]:
def get_bow(words):
    # 辞書何にある単語の出現回数を数える
    bow_id = dictionary.doc2bow(words)
    # 長さを揃えて転置
    bow = matutils.corpus2dense([bow_id], n_words).T
    return bow

x = get_bow(words)
x

array([[1., 1., 1., ..., 0., 0., 0.]], dtype=float32)

In [38]:
y = clf.predict(x)[0]
y

0

In [39]:
for url in urls:
    browser.get(url)
    text = browser.find_element_by_class_name('newsContent_inner').text
    words = preprocessing(text)
    x = get_bow(words)
    y = clf.predict(x)[0]
    print('URL: {}, Label: {}'.format(url, y))
    

URL: https://sandbox.kikagaku.co.jp/blog/post1.html, Label: 0
URL: https://sandbox.kikagaku.co.jp/blog/post2.html, Label: 0
URL: https://sandbox.kikagaku.co.jp/blog/post3.html, Label: 0
URL: https://sandbox.kikagaku.co.jp/blog/post4.html, Label: 1


In [45]:
import requests
import json

slack_url = 'https://hooks.slack.com/services/...'
data = json.dumps({
    'username': 'bot',
    'text': 'テスト'
})
data

'{"username": "bot", "text": "\\u30c6\\u30b9\\u30c8"}'

In [46]:
requests.post(slack_url, data=data)


<Response [200]>